In [1]:
# 사용 모듈
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import types
import pandas as pd
import pickle

In [9]:
# blockchain.csv파일을 읽어와 이를 DataFrame으로 targetData 변수에 저장합니다.
targetData = pd.read_csv("blockchain.csv")

In [10]:
targetData

,0,0.1,1686824250.3353186,Genesis Block,91ee2f941351a761503582a77b51180d8813ded785732ef123b1fbdcd9bce283,0.2
0,1,91ee2f941351a761503582a77b51180d8813ded785732e...,1.686824e+09,[1ca509ef-0495-4ab6-bcb8-2b3cb0fdfb7e]UserID S...,0000095fbc41d81db203698178b4570a455d936feda95e...,1226712
1,2,0000095fbc41d81db203698178b4570a455d936feda95e...,1.686874e+09,[3325a1cb-ba6b-40f3-8fc3-80bfa9ba7207]UserID S...,00000fb8ca9ef2603bcdc5e483fd16d442f5a2fbc4c8e7...,21663
2,3,00000fb8ca9ef2603bcdc5e483fd16d442f5a2fbc4c8e7...,1.686874e+09,[0408f9e6-3937-4af5-9e6c-8c4b9a95d529]UserID S...,00000f3a918517f85f07107d63d9a371d1559305096b25...,1340278
3,4,00000f3a918517f85f07107d63d9a371d1559305096b25...,1.686874e+09,[90cdd30b-6105-4da5-af95-9c36048d085c]UserID S...,0000063f392a7598d69f4059aa3423d4e81625cb5393e8...,996847


In [11]:
# genesisBlock의 데이터를 별도로 저장합니다.
genesisBlock = pd.DataFrame(targetData.columns)

In [12]:
genesisBlock

,0
0,0
1,0.1
2,1686824250.3353186
3,Genesis Block
4,91ee2f941351a761503582a77b51180d8813ded785732e...
5,0.2


In [13]:
# blockchain.csv 데이터가 담긴 데이터프레임의 컬럼 명을 지정합니다.
targetData.columns = ["index", "previousHash", "timestamp", "data", "presentHash", "proof"]

In [14]:
targetData

,index,previousHash,timestamp,data,presentHash,proof
0,1,91ee2f941351a761503582a77b51180d8813ded785732e...,1.686824e+09,[1ca509ef-0495-4ab6-bcb8-2b3cb0fdfb7e]UserID S...,0000095fbc41d81db203698178b4570a455d936feda95e...,1226712
1,2,0000095fbc41d81db203698178b4570a455d936feda95e...,1.686874e+09,[3325a1cb-ba6b-40f3-8fc3-80bfa9ba7207]UserID S...,00000fb8ca9ef2603bcdc5e483fd16d442f5a2fbc4c8e7...,21663
2,3,00000fb8ca9ef2603bcdc5e483fd16d442f5a2fbc4c8e7...,1.686874e+09,[0408f9e6-3937-4af5-9e6c-8c4b9a95d529]UserID S...,00000f3a918517f85f07107d63d9a371d1559305096b25...,1340278
3,4,00000f3a918517f85f07107d63d9a371d1559305096b25...,1.686874e+09,[90cdd30b-6105-4da5-af95-9c36048d085c]UserID S...,0000063f392a7598d69f4059aa3423d4e81625cb5393e8...,996847


In [15]:
# pickle로 저장된 mariaDB 접속 정보를 불러온다.
with open("mariaData.pickle", "rb") as mariaData:
    mariaDbInfo = pickle.load(mariaData)

In [17]:
mariaId = mariaDbInfo.get("mariaId")
mariaPw = mariaDbInfo.get("mariaPw")
mariaIp = mariaDbInfo.get("mariaIp")
mariaPort = mariaDbInfo.get("mariaPort")
mariaDbName = mariaDbInfo.get("mariaDbName")

mariaEngine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(mariaId, mariaPw, mariaIp, mariaPort, mariaDbName))

In [18]:
# ETL 간 사용할 테이블명
tableName = "groupblockdata"

# 전송 처리 향상을 도모 
objectColumns = list(targetData.columns[targetData.dtypes == 'object'])
typeDict={}
maxLen = 500
for i in range(0, len(objectColumns)):
    typeDict[ objectColumns[i] ] = types.VARCHAR(maxLen)

In [19]:
# MariaDB와 oracleDB 양 쪽으로 대상 targetData를 전송. 테이블명 : groupblockdata.
targetData.to_sql(name=tableName, if_exists="replace", con=mariaEngine, dtype=typeDict, index=False)

4

In [20]:
# MariaDB와 oracleDB로부터 groupblockdata 테이블로부터 데이터를 불러와 각각 mariaBlockData와 oracleBlockData 변수에 저장
mariaBlockData = pd.read_sql_query("SELECT * FROM groupblockdata", con = mariaEngine)

In [21]:
mariaBlockData

,index,previousHash,timestamp,data,presentHash,proof
0,1,91ee2f941351a761503582a77b51180d8813ded785732e...,1.686824e+09,[1ca509ef-0495-4ab6-bcb8-2b3cb0fdfb7e]UserID S...,0000095fbc41d81db203698178b4570a455d936feda95e...,1226712
1,2,0000095fbc41d81db203698178b4570a455d936feda95e...,1.686874e+09,[3325a1cb-ba6b-40f3-8fc3-80bfa9ba7207]UserID S...,00000fb8ca9ef2603bcdc5e483fd16d442f5a2fbc4c8e7...,21663
2,3,00000fb8ca9ef2603bcdc5e483fd16d442f5a2fbc4c8e7...,1.686874e+09,[0408f9e6-3937-4af5-9e6c-8c4b9a95d529]UserID S...,00000f3a918517f85f07107d63d9a371d1559305096b25...,1340278
3,4,00000f3a918517f85f07107d63d9a371d1559305096b25...,1.686874e+09,[90cdd30b-6105-4da5-af95-9c36048d085c]UserID S...,0000063f392a7598d69f4059aa3423d4e81625cb5393e8...,996847


In [ ]:
# 두 데이터의 길이가 같다면 targetStr 변수에 True를, 다르다면 False를 저장 
if(mariaBlockData.shape[0] == oracleBlockData.shape[0]):
    targetStr = "True"
else:
    targetStr = "False"

# 만약 targetStr이 True라면, 두 테이블의 presentHash 값을 모두 비교하고,
# 하나라도 다른 해시값이 존재한다면 "온전하지 않은 블록이 존재합니다" 라는 문자열을 return하고 for문을 탈출합니다.
# 모든 presentHash 값이 일치한다면 "온전한 블록으로 구성되어 있습니다." 라는 문자열을 return합니다.
if(targetStr == "True"):
    for i in range(0, mariaBlockData.shape[0]):
        if(oracleBlockData.presentHash[i] != mariaBlockData.presentHash[i]):
            answerStr = "온전하지 않은 블록이 존재합니다."
            break
        else:
            answerStr = "온전한 블록으로 구성되어있습니다."
# 만약 answerStr이 False라면, " 두 DB에 저장된 값이 일치하지 않습니다." 라는 문자열을 return합니다.
else:
    answerStr = "두 DB에 저장된 값이 일치하지 않습니다."